<a href="https://colab.research.google.com/github/vaishnavisharma6/Tensorflow-Practice/blob/main/ODE_using_PINNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Solution of a 2 point boundary value problem using Physics informed Neural Network approach

Given a boundary value problem:
$$
- \frac{d^{2}u}{dx^{2}} = - e ^{(x-1)},    \qquad x \in (0,1)
$$
subject to the conditions:
$$
u(0) = a = 0, \qquad \frac{du}{dx} = b = 1 ,\qquad x = 1
$$

We will solve this differential equation by training a neural network in which loss function will be defined in such a manner that when it is minimized, equation is satisfied automatically.



In [18]:
import os
from IPython.display import clear_output
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np
import random
import matplotlib.pyplot as plt

In [19]:
# Model architecture
def NN():
  model = Sequential()
  model.add(Dense(512, input_shape = (1,), activation = 'relu'))
  model.add(Dense(256, activation = 'relu' ))
  model.add(Dense(1))

  return model

In [20]:
# Define loss function
def lossfunc(x, model):
  
  with tf.GradientTape() as t1:
    t1.watch(x)
    with tf.GradientTape() as t2:
      t2.watch(x)
      u = model(x)
      loss_res = np.exp(x-1)
      loss_bnd = u[0,0]**2 
    dudx = t2.gradient(u,x)
    loss_bnd += (dudx[-1,0] - 1)**2
  du2dx2 = t1.gradient(dudx, x)
  loss_res += -1 * du2dx2
  loss_res = tf.reduce_mean(tf.square(loss_res))

  return  loss_res, loss_bnd

In [21]:
def train(x, lb, epochs):
  res_loss_loc = np.zeros(epochs)
  bnd_loss_loc = np.zeros(epochs)
  loss_loc = np.zeros(epochs)

  tf.keras.backend.clear_session()
  model = NN()
  optimizer = keras.optimizers.Adam()

  for epoch in range(epochs):
    with tf.GradientTape() as tape:
      loss_res, loss_bnd = lossfunc(x = x, model = model)
      loss = loss_res + (lb * loss_bnd)
      loss += sum(model.losses)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    res_loss_loc[epoch] = loss_res.numpy()
    bnd_loss_loc[epoch] = loss_bnd.numpy()
    loss_loc[epoch] = loss.numpy()

  return  res_loss_loc, bnd_loss_loc, loss_loc, model  


In [23]:
epochs = 2000
N =100
x = tf.cast(tf.reshape(tf.linspace(0,1,N), (-1,1)), tf.float32)
lb = 1000.0

res_loss, bnd_loss, loss, model = train(x, lb, epochs)
y = model.predict(x)
print(f" loss_res : {res_loss[-1]}, loss_bnd : {bnd_loss[-1]}, loss_total: {loss[-1]}") 

4/4 [==============================] - 0s 4ms/step
 loss_res : 0.4337002635002136, loss_bnd : 0.0, loss_total: 0.4337002635002136
